In [3]:
import sqlite3
import pandas as pd

## Подключение к БД

In [6]:
conn = sqlite3.connect("data.db")



Выбираем все договоры самой старшей версии с меткой emergency - "Закупка осуществляется вследствие аварии,
 иных чрезвычайных ситуаций природного или техногенного характера, непреодолимой силы,
 принеобходимости срочного медицинского вмешательства, а также для предотвращения угрозы
 возникновения указанных ситуаций"

In [13]:
covid_contracts = pd.read_sql_query(
    """
    SELECT *
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    """,
    conn
)

covid_contracts

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,has_okpd_and_okdp_rows,has_okpd2_rows,is_electronic_place,electronic_place_name,electronic_place_url,electronic_place_publish_date,electronic_place_guid,region_name,contract_reg_number,ver
0,0008f92e-3856-4383-97d2-2fd8b9b71aee,57708503727200082220007,0,1,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,57708503727200082220000,4
1,00203b76-6dc4-4c1d-a191-da8bf4fb296a,57727344543200003930001,0,1,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,57727344543200003930000,1
2,0030cab8-72d3-444e-b6ed-84e7bb44f22f,87729082090200002390001,0,1,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,87729082090200002390000,1
3,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,0,32009471555 Выполнение аварийно-восстановитель...,None,1,1,0,0,...,0,1,None,None,None,None,None,Sankt-Peterburg,87825011824200000490000,1
4,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,56901067107210018890000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,67703695246200001160000,1
3702,ffbdaf3d-2e8d-4305-a550-c713406a61a2,59701039940200000450001,0,1,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,59701039940200000450000,1
3703,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Krasnodarskii_krai,52312210286200000990000,1
3704,ffda639d-aada-44de-974a-4ce9ac345a55,67723011906200009850003,0,0,None,None,0,0,0,0,...,0,1,None,None,None,None,None,Moskva,67723011906200009850000,2


Для дальнейшего анализа необходимо проверить наличие поля с указанием лота

In [14]:
covid_contracts_with_not_null_lot = covid_contracts[covid_contracts["lot_guid"].notnull()]

print(f'Количество "ковидных" договоров: {covid_contracts.shape[0]}')
print(f'Количество "ковидных" договоров с указанием лота: {covid_contracts_with_not_null_lot.shape[0]}')

Количество "ковидных" договоров: 3706
Количество "ковидных" договоров с указанием лота: 1643


Договоры без указания lot_guid, это те где извещение не размещалось в соответствии
с частью 5 статьи 4 Федерального Закона № 223-ФЗ, например https://zakupki.gov.ru/epz/contractfz223/card/contract-info.html?id=9268727


Выберем из базы договоры с лотами

In [35]:
covid_contracts_with_lot = pd.read_sql_query(
    """
    SELECT *
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    """,
    conn
)

covid_contracts_with_lot

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,ignored_purchase,purchase_category_code,centralized,purchase_description,application_supply_needed,application_supply_summ,application_supply_currency_code,application_supply_extra,major_contract_conditions,antimonopoly_decision_taken
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,0,32009471555 Выполнение аварийно-восстановитель...,None,1,1,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,None,None,0,0,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,None,None,0,0,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,None,None,0,0,0,0,...,1,1.0,0,None,NaN,NaN,None,None,None,None
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,None,None,0,0,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,None,None,0,0,0,0,...,1,1.0,0,None,NaN,NaN,None,None,None,None
1537,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,None,None,0,0,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,None,None,0,0,0,0,...,1,1.0,0,None,NaN,NaN,None,None,None,None
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,None,None,0,0,0,0,...,0,NaN,0,None,NaN,NaN,None,None,None,None


Дополним информацией о том, был ли договор завершен или расторгнут

In [39]:
covid_contracts_performance_with_lot = pd.read_sql_query(
    """
    SELECT c.*, pnl.*, pnld.*, cp.*
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN contract_performance cp on cp.guid = (
                    SELECT cp1.guid FROM contract_performance cp1
                    WHERE cp1.contract_registration_number = c.contract_reg_number
                    ORDER BY cp1.is_complete
                    LIMIT 1
                    )
    """,
    conn
)

covid_contracts_performance_with_lot

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,application_supply_extra,major_contract_conditions,antimonopoly_decision_taken,guid,registration_number,contract_registration_number,status,publication_date,placer_inn,is_complete
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,0,32009471555 Выполнение аварийно-восстановитель...,None,1,1,0,0,...,None,None,None,83a3bf65-4e85-4177-a7c8-db0e962bc19f,87825011824200000490007,87825011824200000490000,Размещено,2020-11-16 13:32:29,7825011824,1.0
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
1537,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,None,None,0,0,0,0,...,None,None,None,None,None,None,None,None,None,NaN


Необходимо дополнить выборку информацией о позициях договора

In [38]:
covid_contracts_positions_with_lot = pd.read_sql_query(
    """
    SELECT c.*, pnl.*, pnld.*, cp.*, cpos.*
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_performance cp on cp.guid = (
                    SELECT cp1.guid FROM contract_performance cp1
                    WHERE cp1.contract_registration_number = c.contract_reg_number
                    ORDER BY cp1.is_complete
                    LIMIT 1
                    )
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    """,
    conn
)

covid_contracts_positions_with_lot

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,producer_country,impossible_to_determine_attr,okei_code,okei_name,qty,unit_price,currency_code,exchange_rate,rub_unit_price,source_info
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,0,32009471555 Выполнение аварийно-восстановитель...,None,1,1,0,0,...,None,0,876,Условная единица,1.0,590489.94,RUB,NaN,NaN,None
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,None,None,0,0,0,0,...,None,1,None,None,NaN,2108040.00,RUB,NaN,NaN,None
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,None,None,0,0,0,0,...,None,1,None,None,NaN,NaN,None,NaN,NaN,None
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,None,None,0,0,0,0,...,None,0,796,Штука,18.0,NaN,None,NaN,NaN,None
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,None,None,0,0,0,0,...,None,1,None,None,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,None,None,0,0,0,0,...,None,0,796,Штука,3.0,NaN,None,NaN,NaN,None
2045,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,None,None,0,0,0,0,...,None,0,796,Штука,1.0,NaN,None,NaN,NaN,None
2046,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,None,None,0,0,0,0,...,None,0,796,Штука,220.0,NaN,None,NaN,NaN,None
2047,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,None,None,0,0,0,0,...,None,1,None,None,NaN,NaN,None,NaN,NaN,None


Выгрузим датасет в эксель

In [40]:
covid_contracts_positions_with_lot.to_excel("output.xlsx")